In [12]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [13]:
!pip install moviepy

In [14]:
import torch
import cupy as cp
from moviepy.editor import VideoFileClip
import pandas as pd
import librosa
import scipy.stats
import soundfile as sf
import io
import os
from tqdm import tqdm
import pickle as pk

# Set device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


Statistical Features  
A first easy step is to compute the mean, standard deviation, minimum, maximum, median and quartiles of the frequencies of each signal. This can be done using Numpy and it always brings value to our feature extraction.

In [15]:
def describe_freq(freqs):
    freqs = cp.array(freqs)  # Convert to CuPy array for GPU computation
    mean = cp.mean(freqs)
    std = cp.std(freqs)
    maxv = cp.amax(freqs)
    minv = cp.amin(freqs)
    median = cp.median(freqs)
    skew = scipy.stats.skew(cp.asnumpy(freqs))  # Skew not directly supported in CuPy
    kurt = scipy.stats.kurtosis(cp.asnumpy(freqs))  # Kurtosis not directly supported in CuPy
    q1 = cp.quantile(freqs, 0.25)
    q3 = cp.quantile(freqs, 0.75)
    mode = scipy.stats.mode(cp.asnumpy(freqs))[0][0]  # Mode not directly supported in CuPy
    iqr = cp.subtract(q3, q1)

    return [mean.get(), std.get(), maxv.get(), minv.get(), median.get(), skew, kurt, q1.get(), q3.get(), mode, iqr.get()]

In [16]:
def get_features(x, sr):
    x = torch.tensor(x, device=device)  # Send to GPU
    rmse = torch.mean(torch.tensor(librosa.feature.rms(y=x.cpu().numpy())[0], device=device))
    zcr = torch.mean(torch.tensor(librosa.feature.zero_crossing_rate(x.cpu().numpy())[0], device=device))
    tempo = torch.tensor(librosa.beat.tempo(y=x.cpu().numpy(), sr=sr)[0], device=device)
    mfcc = torch.mean(torch.tensor(librosa.feature.mfcc(y=x.cpu().numpy(), sr=sr), device=device), axis=1)
    spec_cen = torch.mean(torch.tensor(librosa.feature.spectral_centroid(y=x.cpu().numpy(), sr=sr), device=device))
    spectral_bandwidth = torch.mean(torch.tensor(librosa.feature.spectral_bandwidth(y=x.cpu().numpy(), sr=sr), device=device))
    spectral_contrast = torch.mean(torch.tensor(librosa.feature.spectral_contrast(y=x.cpu().numpy(), sr=sr), device=device))
    spectral_flatness = torch.mean(torch.tensor(librosa.feature.spectral_flatness(y=x.cpu().numpy()), device=device))
    spectral_rolloff = torch.mean(torch.tensor(librosa.feature.spectral_rolloff(y=x.cpu().numpy(), sr=sr), device=device))

    features = [rmse, zcr, tempo, spec_cen, spectral_bandwidth, spectral_contrast, spectral_flatness, spectral_rolloff]
    features = [f.item() for f in features] + [mfcc[i].item() for i in range(mfcc.size(0))]  # Convert to list
    return features

In [17]:
def extract_features(file_path):
    try:
        # Load video file
        video_clip = VideoFileClip(file_path)
        audio = video_clip.audio
        fps = audio.fps
        audio_samples = cp.array(list(audio.iter_frames(fps=fps, dtype="float32"))).flatten()
        buffer = io.BytesIO()
        sf.write(buffer, cp.asnumpy(audio_samples), fps, format="wav")
        buffer.seek(0)
        x, sr = librosa.load(buffer, sr=None)
        video_clip.close()
        features = get_features(x, sr)
        return features

    except Exception as e:
        print(f"Error encountered while parsing file: {file_path}, {e}")
        return None

In [18]:
def load_data(real_dir, fake_dir, real_files, fake_files):
    data = []
    columns = ["rmse", "zcr", "tempo", "spectral_centroid", "spectral_bandwidth",
               "spectral_contrast", "spectral_flatness", "spectral_rolloff"] + \
              [f"mfcc{i}" for i in range(1, 21)] + ["label"]

    # Set up progress bar
    total_files = len(real_files) + len(fake_files)
    pbar = tqdm(total=total_files, desc="Processing files", unit="file")

    # Process real audio files
    for file_name in real_files:
        file_path = os.path.join(real_dir, file_name)
        features = extract_features(file_path)
        if features is not None:
            features.append(0)  # Label: 0 for REAL
            data.append(features)
        pbar.update(1)

    # Process fake audio files
    for file_name in fake_files:
        file_path = os.path.join(fake_dir, file_name)
        features = extract_features(file_path)
        if features is not None:
            features.append(1)  # Label: 1 for FAKE
            data.append(features)
        pbar.update(1)

    pbar.close()
    df = pd.DataFrame(data, columns=columns)
    return df


In [19]:
real_audio_dir = r"/content/drive/MyDrive/SIH2024_DATASET/FAKE"
fake_audio_dir = r"/content/drive/MyDrive/SIH2024_DATASET/REAL"

In [20]:
with open(
    r"/content/drive/MyDrive/SIH2024_DATASET/real_files.pkl",
    "rb",
) as f:
    real_files = pk.load(f)

with open(
    r"/content/drive/MyDrive/SIH2024_DATASET/fake_files.pkl",
    "rb",
) as f:
    fake_files = pk.load(f)

In [ ]:
df = load_data(real_audio_dir, fake_audio_dir, fake_files, real_files)


Processing files:   0%|          | 0/119146 [00:00<?, ?file/s]WARNING:py.warnings:<ipython-input-16-a7fd88b90afb>:5: FutureWarning: librosa.beat.tempo
	This function was moved to 'librosa.feature.rhythm.tempo' in librosa version 0.10.0.
	This alias will be removed in librosa version 1.0.
  tempo = torch.tensor(librosa.beat.tempo(y=x.cpu().numpy(), sr=sr)[0], device=device)


Processing files:   0%|          | 35/119146 [04:21<202:28:58,  6.12s/file]

In [ ]:
df.tail()

In [ ]:
# for file in file_names:

#     clean_file = file.split("/")[-1]
#     video_clip = VideoFileClip(file)
#     audio = video_clip.audio
#     fps = audio.fps
#     audio_samples = np.array(list(audio.iter_frames(fps=fps, dtype="float32"))).flatten()
#     buffer = io.BytesIO()
#     sf.write(buffer, audio_samples, fps, format='wav')
#     buffer.seek(0)
#     x, sr = librosa.load(buffer, sr=None)
#     label = json.load(open("train_sample_videos/metadata.json"))[clean_file]['label']
#     new_row = pd.DataFrame([[clean_file] + get_features(x, sr) + [label]], columns=column_ames)
#     df = pd.concat([df, new_row], ignore_index=True)

In [ ]:
df.to_csv( "/content/drive/MyDrive/SIH2024_DATASET/full_features.csv", index=False)